In [1]:
import json
from elasticsearch import Elasticsearch

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
!ls -ltrh ../data-extraction

total 13264
-rw-r--r--@ 1 amoghkulkarni  staff     0B Aug 31 21:12 example.ipynb
drwxr-xr-x@ 5 amoghkulkarni  staff   160B Sep  1 16:46 html_docs
-rw-r--r--@ 1 amoghkulkarni  staff    11K Sep  3 08:06 failed_phone_list.md
-rw-r--r--@ 1 amoghkulkarni  staff   4.3M Sep 13 10:33 mobile_specifications_data.json
-rw-r--r--@ 1 amoghkulkarni  staff   281K Sep 14 10:10 gsm-website-extraction-poc.ipynb
-rw-r--r--@ 1 amoghkulkarni  staff   1.0M Sep 14 10:10 errored_mobile_specifications_data.json


In [2]:
with open('../data-extraction/mobile_specifications_data.json', 'r') as fp:
    mobile_specs_data = json.load(fp)

{'dizo-star-500': {'General': {'Brand': 'Dizo',
   'Model': 'Star 500',
   'Price in India': '₹1,425',
   'Release date': '8th July 2021',
   'Launched in India': 'Yes',
   'Form factor': 'Bar',
   'Dimensions (mm)': '132.50 x 55.40 x 12.00',
   'Weight (g)': '108.00',
   'Battery capacity (mAh)': '1900',
   'Colours': 'Black, Green, Silver'},
  'Display': {'Screen size (inches)': '2.80',
   'Touchscreen': 'No',
   'Resolution': '320x240 pixels'},
  'Hardware': {'RAM': '32MB',
   'Internal storage': '32MB',
   'Expandable storage': 'Yes',
   'Expandable storage up to (GB)': '64'},
  'Camera': {'Rear camera': '0.3-megapixel',
   'No. of Rear Cameras': '1',
   'Front camera': 'No'},
  'Connectivity': {'Number of SIMs': '2',
   'Active 4G on both SIM cards': 'No',
   'SIM Type': 'Micro-SIM',
   'GSM/CDMA': 'GSM'}},
 'dizo-star-300': {'General': {'Brand': 'Dizo',
   'Model': 'Star 300',
   'Price in India': '₹1,049',
   'Release date': '8th July 2021',
   'Launched in India': 'Yes',
   'Fo

In [3]:
from tqdm.auto import tqdm

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sentence_transformers import SentenceTransformer

In [21]:
!pip freeze | grep -i sentence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


sentence-transformers==2.7.0


In [6]:
import numpy as np

In [4]:
np.__version__

'1.26.4'

In [8]:
model = SentenceTransformer("all-mpnet-base-v2")

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
len(model.encode("dizo-star-500"))

768

In [30]:
es_client = Elasticsearch('http://localhost:9200')

In [31]:
es_client.info()

ObjectApiResponse({'name': '3ccafbf6ba75', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'PjY-6oAsRiykpDnJfsyPLg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [41]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "company": {"type": "text"},
            "name": {"type": "text"},
            "specifications": {"type": "keyword"} ,
            "specification_vector": {"type": "dense_vector", "dims": 96, "index": True, "similarity": "cosine"},
            "name_vector": {"type": "dense_vector", "dims": 96, "index": True, "similarity": "cosine"}
        }
    }
}

In [64]:
index_name = "mobile-specifications"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mobile-specifications'})

In [18]:
mobile_specs_rows = []

for each_mobile_company in tqdm(mobile_specs_data):
    for each_mobile in mobile_specs_data[each_mobile_company]:
        temp_mobile_dictionary = {
            'company': each_mobile_company,
            'name': each_mobile,
            'specifications': json.dumps(mobile_specs_data[each_mobile_company][each_mobile])
        }
        mobile_specs_rows.append(temp_mobile_dictionary)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 2050.52it/s]


### This was where I screwed up the logic of dictionary references

In [20]:
mobile_specs_rows = []

for each_mobile_company in tqdm(mobile_specs_data):
    temp_mobile_dictionary = {
        'company': each_mobile_company
    }
    for each_mobile in mobile_specs_data[each_mobile_company]:
        temp_mobile_dictionary.update({ 'name': each_mobile })
        temp_mobile_dictionary.update({ 'specifications': json.dumps(mobile_specs_data[each_mobile_company][each_mobile]) })
        mobile_specs_rows.append(temp_mobile_dictionary)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 3065.81it/s]


In [136]:
company = '10-or-phones'
temp_dict = {
    'company': company
}

In [137]:
mob_1 = '10.or-e'

In [138]:
mob_2 = '10.or-g2'

In [139]:
temp_dict.update({ 'name': mob_1 })
temp_dict.update({ 'specifications': json.dumps(mobile_specs_data[company][mob_1]) })

In [140]:
new_list = []

In [141]:
new_list.append(temp_dict)

In [142]:
temp_dict

{'company': '10-or-phones',
 'name': '10.or-e',
 'specifications': '{"General": {"Brand": "10.or", "Model": "E", "Price in India": "\\u20b94,999", "Release date": "September 2017", "Form factor": "Touchscreen", "Battery capacity (mAh)": "4000", "Removable battery": "No", "Colours": "Aim Gold, Beyond Black"}, "Display": {"Screen size (inches)": "5.50", "Touchscreen": "Yes", "Resolution": "1080x1920 pixels"}, "Hardware": {"Processor": "octa-core", "Processor make": "Qualcomm Snapdragon 430", "RAM": "2GB", "Internal storage": "16GB", "Expandable storage": "Yes", "Expandable storage type": "microSD", "Expandable storage up to (GB)": "128"}, "Camera": {"Rear camera": "13-megapixel", "Rear flash": "Yes", "Front camera": "5-megapixel", "Front flash": "Yes"}, "Software": {"Operating system": "Android 7.1.2"}, "Connectivity": {"Wi-Fi": "Yes", "Wi-Fi standards supported": "802.11 b/g/n", "GPS": "Yes", "Bluetooth": "Yes", "NFC": "No", "Infrared": "No", "USB OTG": "No", "FM": "No", "Number of SIMs

In [144]:
temp_dict.update({ 'name': mob_2 })
temp_dict.update({ 'specifications': json.dumps(mobile_specs_data[company][mob_2]) })

In [145]:
temp_dict

{'company': '10-or-phones',
 'name': '10.or-g2',
 'specifications': '{"General": {"Brand": "10.or", "Model": "G2", "Price in India": "\\u20b97,999", "Release date": "28th June 2019", "Launched in India": "Yes", "Form factor": "Touchscreen", "Battery capacity (mAh)": "5000", "Fast charging": "Proprietary", "Colours": "Charcoal Black, Twilight Blue"}, "Display": {"Screen size (inches)": "6.18", "Touchscreen": "Yes", "Resolution": "1080x2246 pixels", "Protection type": "Gorilla Glass", "Aspect ratio": "19:9"}, "Hardware": {"Processor": "octa-core", "Processor make": "Qualcomm Snapdragon 636", "RAM": "4GB", "Internal storage": "64GB", "Expandable storage": "Yes", "Expandable storage type": "microSD", "Expandable storage up to (GB)": "256", "Dedicated microSD slot": "Yes"}, "Camera": {"Rear camera": "16-megapixel + 5-megapixel", "Rear autofocus": "Yes", "Rear flash": "Dual LED", "Front camera": "12-megapixel"}, "Software": {"Operating system": "Android"}, "Connectivity": {"Wi-Fi": "Yes", "W

In [146]:
new_list

[{'company': '10-or-phones',
  'name': '10.or-g2',
  'specifications': '{"General": {"Brand": "10.or", "Model": "G2", "Price in India": "\\u20b97,999", "Release date": "28th June 2019", "Launched in India": "Yes", "Form factor": "Touchscreen", "Battery capacity (mAh)": "5000", "Fast charging": "Proprietary", "Colours": "Charcoal Black, Twilight Blue"}, "Display": {"Screen size (inches)": "6.18", "Touchscreen": "Yes", "Resolution": "1080x2246 pixels", "Protection type": "Gorilla Glass", "Aspect ratio": "19:9"}, "Hardware": {"Processor": "octa-core", "Processor make": "Qualcomm Snapdragon 636", "RAM": "4GB", "Internal storage": "64GB", "Expandable storage": "Yes", "Expandable storage type": "microSD", "Expandable storage up to (GB)": "256", "Dedicated microSD slot": "Yes"}, "Camera": {"Rear camera": "16-megapixel + 5-megapixel", "Rear autofocus": "Yes", "Rear flash": "Dual LED", "Front camera": "12-megapixel"}, "Software": {"Operating system": "Android"}, "Connectivity": {"Wi-Fi": "Yes",

In [147]:
new_list.append(temp_dict)
new_list

[{'company': '10-or-phones',
  'name': '10.or-g2',
  'specifications': '{"General": {"Brand": "10.or", "Model": "G2", "Price in India": "\\u20b97,999", "Release date": "28th June 2019", "Launched in India": "Yes", "Form factor": "Touchscreen", "Battery capacity (mAh)": "5000", "Fast charging": "Proprietary", "Colours": "Charcoal Black, Twilight Blue"}, "Display": {"Screen size (inches)": "6.18", "Touchscreen": "Yes", "Resolution": "1080x2246 pixels", "Protection type": "Gorilla Glass", "Aspect ratio": "19:9"}, "Hardware": {"Processor": "octa-core", "Processor make": "Qualcomm Snapdragon 636", "RAM": "4GB", "Internal storage": "64GB", "Expandable storage": "Yes", "Expandable storage type": "microSD", "Expandable storage up to (GB)": "256", "Dedicated microSD slot": "Yes"}, "Camera": {"Rear camera": "16-megapixel + 5-megapixel", "Rear autofocus": "Yes", "Rear flash": "Dual LED", "Front camera": "12-megapixel"}, "Software": {"Operating system": "Android"}, "Connectivity": {"Wi-Fi": "Yes",

### screw up ends

In [33]:
for each_mobile in tqdm(mobile_specs_rows):
    each_mobile['specification_vector'] = model.encode(each_mobile['specifications']).tolist()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [27:37<00:00,  1.26it/s]


In [25]:
for each_mobile in tqdm(mobile_specs_rows):
    each_mobile['name_vector'] = model.encode(each_mobile['name']).tolist()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [03:58<00:00,  8.80it/s]


In [26]:
for doc in tqdm(mobile_specs_rows):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [00:19<00:00, 107.98it/s]


In [9]:
search_term = 'iphone-16-pro'
vector_search_term = model.encode(search_term)

In [15]:
knn_query = {
    "field": "specification_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [16]:
index_name = "mobile-specifications"

In [17]:
res = es_client.search(index=index_name,knn=knn_query, source=["name", "specifications", "company"], size=10)

In [18]:
res['hits']

{'total': {'value': 5, 'relation': 'eq'},
 'max_score': 0.8278189,
 'hits': [{'_index': 'mobile-specifications',
   '_id': 'GcbuXJIBzLYN4yyb7iVc',
   '_score': 0.8278189,
   '_source': {'name': 'iphone-16-pro-max',
    'company': 'apple-phones',
    'specifications': '{"General": {"Brand": "Apple", "Model": "iPhone 16 Pro Max", "Price in India": "\\u20b9164,900", "Release date": "9th September 2024", "Launched in India": "Yes", "Form factor": "Touchscreen", "Dimensions (mm)": "163.00 x 77.60 x 8.25", "Weight (g)": "227.00", "IP rating": "IP68", "Removable battery": "No", "Wireless charging": "Yes", "Wireless Charging Type": "Qi 2, MagSafe", "Colours": "Black Titanium, Natural Titanium, White Titanium, Desert Titanium"}, "Display": {"Refresh Rate": "120 Hz", "Screen size (inches)": "6.90", "Touchscreen": "Yes", "Resolution": "1320x2868 pixels", "Pixels per inch (PPI)": "460"}, "Hardware": {"Processor": "hexa-core", "Processor make": "Apple A18 Pro", "RAM": "8GB", "Internal storage": "25

In [94]:
es_client.cat.count(index=index_name, params={"format": "json"})

/var/folders/zw/mqwlds_s7ml5_fvqptkb174h0000gn/T/ipykernel_87975/1282795516.py:1: DeprecationWarning: The 'params' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.cat.count(index=index_name, params={"format": "json"})


ListApiResponse([{'epoch': '1727149402', 'timestamp': '03:43:22', 'count': '0'}])

In [ ]:
res["hits"]

## Mobile name vector evaluation

#### Encoding mobile name for searching


In [154]:
mobile_specs_rows[0].keys()

dict_keys(['company', 'name', 'specifications', 'name_vector'])

In [152]:
for each_mobile in tqdm(mobile_specs_rows):
    each_mobile['name_vector'] = model.encode(each_mobile['name']).tolist()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [03:55<00:00,  8.91it/s]


In [153]:
len(mobile_specs_rows[0]['name_vector'])

768

In [69]:
for doc in mobile_specs_rows[:1]:
    print(doc.keys())

dict_keys(['company', 'name', 'specifications', 'specification_vector', 'name_vector'])


In [31]:
def elastic_search_knn(vector, vector_field="name_vector"):
    """Return elastic search results for given vector."""
    knn_query = {
        "field": vector_field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        'knn': knn_query,
        '_source': ["name", "company", "specifications"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [32]:
search_term = 'azus-zenfone'
vector_search_term = model.encode(search_term)
zenfone_context = elastic_search_knn(vector_search_term)

In [52]:
prompt_template = """
You are mobile phone expert. Answer all the specifications of PHONE using the CONTEXT provided from SPECFICATIONS database.
The specifications in the CONTEXT is a json string. Convert the json string to readable format before printing out the answer.
Use only the specifications given in the context to print the specs of the device.
Do not number the specifications passed.
Print only the first result in a markdown format where subsections are indented in readable format.


PHONE: {phone}

CONTEXT:
{context}
""".strip()

In [34]:
phone = "asuz-zenfone"

In [35]:
context = ""

for each in zenfone_context:
    context = context + f"\n\nname: {each['name']}\ncompany: {each['company']}\nspecifications: {each['specifications']}"

In [176]:
print(prompt_template.format(phone=phone, context=context))

You are mobile phone expert. Answer all the specifications of PHONE using the CONTEXT provided from SPECFICATIONS database.
The specifications in the CONTEXT is a json string. Convert the json string to readable format before printing out the answer.
Use only the specifications given in the context to print the specs of the device.


PHONE: asuz-zenfone

CONTEXT:


name: asus-zenfone-10
company: asus-phones
specifications: {"General": {"Brand": "Asus", "Model": "Zenfone 10", "Release date": "29th June 2022", "Launched in India": "No", "Form factor": "Touchscreen", "Dimensions (mm)": "146.50 x 68.10 x 9.40", "Weight (g)": "172.00", "IP rating": "IP68", "Battery capacity (mAh)": "5000", "Removable battery": "No", "Fast charging": "Proprietary", "Wireless charging": "Yes", "Colours": "Aurora Green, Comet White, Eclipse Red, Midnight Black, Starry Blue"}, "Display": {"Refresh Rate": "120 Hz", "Resolution Standard": "HD+", "Screen size (inches)": "5.90", "Touchscreen": "Yes", "Resolution": 

In [177]:
!source ~/.zshrc

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
from huggingface_hub import InferenceClient

In [37]:
import os

In [38]:
HF_API_TOKEN = os.getenv('HF_API_TOKEN')
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token=HF_API_TOKEN)

In [53]:
prompt = prompt_template.format(phone=phone, context=context)

In [54]:
messages = [{"role": "user", "content": prompt}]

In [55]:
response = client.chat_completion(messages, max_tokens=1000)

In [57]:
def llm(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat_completion(messages, max_tokens=1000)
    
    return response.choices[0].message.content

In [59]:
print(llm(prompt))

PHONE: asus-zenfone-10

Display:
- Resolution Standard: HD+
- Screen size: 5.90 inches
- Refresh Rate: 120 Hz
- Protection type: Gorilla Glass
- Touchscreen: Yes

Camera:
- Rear camera: 50-megapixel + 13-megapixel
- Number of Rear Cameras: 2
- Front camera: 32-megapixel
- Lens Type (Second Rear Camera): Ultra Wide-Angle

Battery:
- Battery capacity (mAh): 5000
- Removable battery: No
- Fast charging: Proprietary
- Wireless charging: Yes

IP rating: IP68

Connectivity:
- Wi-Fi: Yes
- Wi-Fi standards supported: 802.11 a/b/g/n/ac/ax
- Bluetooth: Yes, v 5.30
- NFC: Yes
- USB Type-C: Yes
- Headphones: 3.5mm
- Number of SIMs: 2
- Active 4G on both SIM cards: Yes
- SIM Type: Nano-SIM

Hardware:
- OS: Android 13
- Skin: Asus ZenUI
- Processor: octa-core
- Processor make: Snapdragon 8 Gen 2
- RAM: 8GB
- Internal storage: 128GB

Sensors:
- Fingerprint sensor: Yes
- Compass/ Magnetometer: Yes
- Proximity sensor: Yes
- Accelerometer: Yes
- Ambient light sensor: Yes
- Gyroscope: Yes

Weight (g): 17

In [67]:
def get_mobile_name_results(mobile_name_statement):
    """Return the ES results of the mobile phone asked by user."""
    name_vector = model.encode(mobile_name_statement)
    mobile_context = elastic_search_knn(name_vector)

    return mobile_context

In [70]:
def build_prompt(phone_specifications, mobile_context):
    """Build the context using search results and return the prompt."""
    prompt_template = """
You are mobile phone expert. Answer all the specifications of PHONE using the CONTEXT provided from SPECFICATIONS database.
The specifications in the CONTEXT is a json string. Convert the json string to readable format before printing out the answer.
Use only the specifications given in the context to print the specs of the device.
Do not number the specifications passed.
Print only the first result in a markdown format where subsections are indented in readable format.


PHONE: {phone}

CONTEXT:
{context}
""".strip()

    context = ""

    for each in mobile_context:
        context = context + f"\n\nname: {each['name']}\ncompany: {each['company']}\nspecifications: {each['specifications']}"

    prompt = prompt_template.format(phone=phone_specifications, context=context)
    return prompt

In [64]:
def llm(prompt):
    """Return LLM generation based on context."""
    messages = [{"role": "user", "content": prompt}]
    response = client.chat_completion(messages, max_tokens=1000)
    
    return response.choices[0].message.content

In [71]:
def rag(mobile_phone_statement):
    """Run the RAG flow for the given mobile phone statement."""
    mobile_name_context = get_mobile_name_results(mobile_phone_statement)
    prompt = build_prompt(mobile_phone_statement, mobile_name_context)
    answer = llm(prompt)

    return answer

In [72]:
print(rag('Tell me the specifications of iphone 15'))

PHONE: Specifications of iPhone 15

The iPhone 15, expected to be released in September 2023, boasts a wide range of impressive features upon its unveiling. Here are its detailed specifications as outlined by the CONTEXT provided by the SPECFICATIONS database.

General:
- Brand: Apple
- Model: iPhone 15
- Price in India: ₹69,900
- Release date: 12th September 2023
- Launched in India: Yes
- Form factor: Touchscreen
- Dimensions (mm): 147.60 x 71.60 x 7.80
- Weight (g): 171.00
- IP rating: IP68
- Removable battery: No
- Wireless charging: Yes
- Colours: Black, Blue, Green, Pink, Yellow

Display:
- Refresh Rate: 60 Hz
- Screen size (inches): 6.10
- Touchscreen: Yes
- Resolution: 1179x2556 pixels
- Pixels per inch (PPI): 460

Hardware:
- Processor: hexa-core
- Processor make: Apple A16 Bionic
- RAM: 6GB
- Internal storage: 128GB, 256GB, 512GB
- Expandable storage: No

Camera:
- Rear camera: 48-megapixel (f/1.6) + 12-megapixel (f/2.4)
- No. of Rear Cameras: 2
- Front camera: 12-megapixel (

## Specifications vector evaluation

    - This is continued in a new file

In [73]:
mobile_specs_vector_data = mobile_specs_rows.copy()

In [74]:
mobile_specs_vector_data[1].keys()

dict_keys(['company', 'name', 'specifications', 'name_vector'])

In [75]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "company": {"type": "text"},
            "name": {"type": "text"},
            "specifications": {"type": "keyword"} ,
            "specification_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
            "name_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [76]:
index_name = "mobile-specifications"
es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mobile-specifications'})

In [77]:
for each_mobile in tqdm(mobile_specs_vector_data):
    each_mobile['specification_vector'] = model.encode(each_mobile['specifications']).tolist()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [29:12<00:00,  1.20it/s]


In [79]:
mobile_specs_vector_data[0].keys()

dict_keys(['company', 'name', 'specifications', 'name_vector', 'specification_vector'])

In [78]:
import pickle

In [80]:
with open('mobile_specs_vector_data.pkl', 'wb') as fp:
    pickle.dump(mobile_specs_vector_data, fp)

In [82]:
with open('mobile_specs_vector_data.pkl', 'rb') as fp:
    bkp_mobile_specs_vector_data = pickle.load(fp)

In [8]:
import json
from elasticsearch import Elasticsearch

In [9]:
with open('../data-extraction/mobile_specifications_data.json', 'r') as fp:
    mobile_specs_data = json.load(fp)

In [13]:
import spacy

In [15]:
!python -m spacy download en_core_web_sm

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [16]:
nlp = spacy.load('en_core_web_sm')

In [20]:
specs_row = mobile_specs_rows[0]['specifications']

In [21]:
vectors = nlp(specs_row)

In [23]:
tokens = [token.lemma_ for token in vectors]

In [27]:
some_vector_list = vectors.vector.tolist()

In [63]:
for each_mobile in tqdm(mobile_specs_rows):
    doc = nlp(each_mobile['specifications'])
    each_mobile['specification_vector'] = doc.vector

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [02:08<00:00, 16.28it/s]


In [34]:
mobile_specs_rows[0].keys()

dict_keys(['company', 'name', 'specifications', 'specification_vector'])

In [65]:
for doc in tqdm(mobile_specs_rows):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2096/2096 [00:12<00:00, 166.60it/s]


In [40]:
len(vectors.vector)

96

In [66]:
search_term = 'samsung-galaxy-quantum-5'
vector_search_term = nlp(search_term)

In [67]:
knn_query = {
    "field": "specification_vector",
    "query_vector": vector_search_term.vector,
    "k": 5,
    "num_candidates": 10000
}

In [56]:
index_name = "mobile-specifications"

In [68]:
res = es_client.search(index=index_name,knn=knn_query, source=["name", "specifications", "company"], size=10)

In [70]:
search_query = {
    'knn': knn_query,
    '_source': ["name", "company", "specifications"]
}

In [71]:
res = es_client.search(index=index_name, body=search_query)

In [72]:
res["hits"]

{'total': {'value': 5, 'relation': 'eq'},
 'max_score': 0.72518784,
 'hits': [{'_index': 'mobile-specifications',
   '_id': 'NhewYJIBdHrq0v253cbH',
   '_score': 0.72518784,
   '_source': {'name': 'redmi-k50-ultra',
    'company': 'redmi-phones',
    'specifications': '{"General": {"Brand": "Redmi", "Model": "K50 Ultra", "Release date": "11th August 2022", "Launched in India": "No", "Battery capacity (mAh)": "5,000", "Removable battery": "No", "Wireless charging": "No", "Colours": "Black, Blue, and Silver"}, "Camera": {"Rear camera": "108-megapixel + 8-megapixel + 2-megapixel", "No. of Rear Cameras": "3", "Front camera": "20-megapixel", "No. of Front Cameras": "1"}}'}},
  {'_index': 'mobile-specifications',
   '_id': '1BewYJIBdHrq0v250MM1',
   '_score': 0.71979403,
   '_source': {'name': 'lg-k92-5g',
    'company': 'lg-phones',
    'specifications': '{"General": {"Brand": "LG", "Model": "K92 5G", "Release date": "30th October 2020", "Launched in India": "No", "Battery capacity (mAh)": "

In [76]:
!pip freeze | grep -i pandas

pandas==2.2.3


In [ ]:
!